# **Practica 2**
#### _Alberto García Doménech - Pablo Daurell Marina_ (Grupo 10)
***

## Parte 1 (Análisis de sentimiento)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("yelp_labelled.txt", sep='\t', names=['Review', 'Valoration'])
data = df.to_numpy()[:, 0]
target = df.to_numpy()[:, 1]

In [4]:
for i in range(10):
    print(data[i], target[i])

Wow... Loved this place. 1
Crust is not good. 0
Not tasty and the texture was just nasty. 0
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it. 1
The selection on the menu was great and so were the prices. 1
Now I am getting angry and I want my damn pho. 0
Honeslty it didn't taste THAT fresh.) 0
The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer. 0
The fries were great too. 1
A great touch. 1


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.25, random_state=0)

In [6]:
print("Training size: ", train_data.size)
print("Test size: ", test_data.size)

Training size:  750
Test size:  250


In [7]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data.any():
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

## A) Bolsas de palabras

In [8]:
from sklearn.feature_extraction.text import CountVectorizerb
from sklearn.feature_extraction.text import TfidfTransformer

### 1. Bolsa de palabras binaria y n-gramas (1,1) 

Creamos una bolsa de palabras con monogramas (solo tenemos en cuenta la probabilidad de una palabra, independientemente del resto) y que tenga en cuenta si una palabra aprece o no:

In [23]:
binary_monogram_vectorizer = CountVectorizer(stop_words='english', binary=True, ngram_range = (1,1)) 
train_vector_data = vectorizer.fit_transform(train_data)

feature_names = vectorizer.get_feature_names()
print(len(feature_names))

1510


### 2. Bolsa de palabras con TF-IDF y n-gramas (1,1)

Creamos una bolsa de palabras con monogramas (solo tenemos en cuenta la probabilidad de una palabra, independientemente del resto) y que tenga en cuenta la frecuencia de aparición:

In [15]:
vectorizer = CountVectorizer(stop_words='english', binary=False, ngram_range = (1,1))
train_vector_data = vectorizer.fit_transform(train_data)

feature_names = vectorizer.get_feature_names()
print(len(feature_names))

1510


Una vez generada la bolsa de palabras, convertimos los valores de frecuencia a TF-IDF:

In [16]:
tfider = TfidfTransformer()
train_preprocessed = tfider.fit_transform(train_vector_data)
print(train_preprocessed.shape)

(750, 1510)


Ajustamos los datos de test al modelo de bolsa de palabra que acabamos de crear:

In [17]:
test_vector_data = vectorizer.transform(test_data)
test_preprocessed = tfider.transform(test_vector_data)
print(test_preprocessed.shape)

(250, 1510)


In [18]:
write_terms(feature_names, train_data, train_preprocessed, 10)
write_terms(feature_names, train_data, train_preprocessed, 100)

Mensaje 10 : The cashew cream sauce was bland and the vegetables were undercooked.
Mensaje 10 vectorizado: ['bland' 'cashew' 'cream' 'sauce' 'undercooked' 'vegetables'] 

Mensaje 100 : Now this dish was quite flavourful.
Mensaje 100 vectorizado: ['dish' 'flavourful' 'quite'] 



### 3. Bolsa de palabras binarias y n-gramas (1,2)

In [81]:
binary_bigram_vectorizer = CountVectorizer(stop_words='english', binary=True, ngram_range = (1,2))
train_vector_data = vectorizer.fit_transform(X_train)
feature_names = vectorizer.get_feature_names()
print(len(feature_names))

4347


### 4. Bolsa de palabras con TF-IDF y n-gramas (1,2)

In [85]:
tfidf_bigram_vectorizer = CountVectorizer(stop_words='english', binary=False ,ngram_range = (1,2))
train_vector_data = vectorizer.fit_transform(X_train)
feature_names = vectorizer.get_feature_names()
print(len(feature_names))

4347


In [ ]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data.any():
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [ ]:
write_terms(feature_names, X_train, train_vector_data, 1)